In [31]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time, random, math
from datetime import datetime
from IPython.display import clear_output
from converter import *
import pandas as pd
import numpy as np
from IPython.display import display
import pickle, os
from IPython.utils import io
with io.capture_output() as captured:
    import translators as ts

In [49]:
def handle_rads(line):
    line=line.replace('\n','')
    meanings,mnemonics=line.split('.',1)
    meanings = (lambda x: [i.lower() for i in x])(meanings.split(','))
    for i in range(len(meanings)):
        if meanings[i][0]==' ':
            meanings[i]=meanings[i][1:]
        if meanings[i][-1]==' ':
            meanings[i]=meanings[i][:-1]
    if mnemonics[0]==' ':
        mnemonics=mnemonics[1:]
    return meanings,mnemonics
def handle_not_rads(line):
    line=line.replace('\n','')
    meanings,readings,mnemonics = line.split(".", 2)
    meanings = (lambda x: [i.lower() for i in x])(meanings.split(','))
    readings = readings.split(',')
    for i in range(len(meanings)):
        if meanings[i][0]==' ':
            meanings[i]=meanings[i][1:]
        if meanings[i][-1]==' ':
            meanings[i]=meanings[i][:-1]
    if mnemonics[0]==' ':
        mnemonics=mnemonics[1:]
    for i in range(len(readings)):
        if readings[i][0]==' ':
            readings[i]=readings[i][1:]
        if readings[i][-1]==' ':
            readings[i]=readings[i][:-1]
    return meanings,readings,mnemonics

    
def FIT(LVL,DIC_RAD={},DIC_KAN={},DIC_VOC={},DIC_ALL={},DIC_NAMES={},STAGE=0):
    RADS_DATA,KANS_DATA,VOCS_DATA={},{},{}
    
    with open('rads/lvl'+str(LVL)+'/data.txt', 'r') as file:
        data = file.readlines()
    for i in data:
        meanings,mnemonics=handle_rads(i)
        RADS_DATA[meanings[0]] = [meanings, mnemonics]
    
    with open('kans/lvl'+str(LVL)+'/data.txt', 'r') as file:
        data = file.readlines()
    for i in data:
        meanings,readings,mnemonics = handle_not_rads(i)
        KANS_DATA[meanings[0]] = [meanings,readings, mnemonics]

    with open('vocs/lvl'+str(LVL)+'/data.txt', 'r') as file:
        data = file.readlines()
    for i in data:
        meanings,readings,mnemonics = handle_not_rads(i)
        VOCS_DATA[meanings[0]] = [meanings,readings, mnemonics]
    
    for dirname, _, filenames in os.walk('rads/lvl'+str(LVL)):
        for filename in filenames:
                filepath = os.path.join(dirname, filename)
                if (filename!='.DS_Store')and(filename!='data.txt')and(('rad',filename[:-4].lower()) not in DIC_NAMES):
                    #stage, pic, meaning, mnemonics='obvious', lvl=0, previous_time=time.time()
                    item=rad(LVL,filepath.lower(),RADS_DATA[filename[:-4].lower()][0],RADS_DATA[filename[:-4].lower()][1],STAGE)
                    DIC_RAD[LVL]=DIC_RAD.get(LVL,[])+[item]
                    DIC_ALL[LVL]=DIC_ALL.get(LVL,[])+[item]
                    DIC_NAMES[('rad',filename[:-4].lower())]=item

    for dirname, _, filenames in os.walk('kans/lvl'+str(LVL)):
        for filename in filenames:
                filepath = os.path.join(dirname, filename)
                if (filename!='.DS_Store')and(filename!='data.txt')and(('kan',filename[:-4].lower()) not in DIC_NAMES):
                    data=KANS_DATA[filename[:-4].lower()]
                    #stage, pic, meanings, readings, mnemonics='obvious', lvl=0, previous_time=time.time()
                    item=kan(LVL,filepath.lower(),data[0],data[1],data[2],STAGE)
                    DIC_KAN[LVL]=DIC_KAN.get(LVL,[])+[item]
                    DIC_ALL[LVL]=DIC_ALL.get(LVL,[])+[item]
                    DIC_NAMES[('kan',filename[:-4].lower())]=item

    for dirname, _, filenames in os.walk('vocs/lvl'+str(LVL)):
        for filename in filenames:
                filepath = os.path.join(dirname, filename)
                if (filename!='.DS_Store')and(filename!='data.txt')and(('voc',filename[:-4].lower()) not in DIC_NAMES):
                    data=VOCS_DATA[filename[:-4].lower()]
                    #stage, pic, meanings, readings, mnemonics='obvious',lvl=0,previous_time=time.time()
                    item=voc(LVL,filepath.lower(),data[0],data[1],data[2],STAGE)
                    DIC_VOC[LVL]=DIC_VOC.get(LVL,[])+[item]
                    DIC_ALL[LVL]=DIC_ALL.get(LVL,[])+[item]
                    DIC_NAMES[('voc',filename[:-4].lower())]=item

    return DIC_RAD,DIC_KAN,DIC_VOC,DIC_ALL,DIC_NAMES

In [40]:
class rad:
    def __init__(self, lvl, pic, meaning, mnemonics='obvious', stage=0, previous_time=time.time()):
        self.previous_time=previous_time
        self.meaning=meaning #LIST
        self.stage=stage
        self.lvl=lvl
        self.pic=pic
        self.mnemonics=mnemonics 
        self.ind_meaning=0
        self.ind_reading=1 
        self.type="RADICAL"   

    def show(self,message,translate=False):
        photo = open(self.pic, 'rb')
        bot.send_message(message.chat.id, 'meaning: '+', '.join(self.meaning))
        bot.send_photo(message.chat.id, photo)
        bot.send_message(message.chat.id, 'mnemonics: '+self.mnemonics)
        if translate:
            bot.send_message(message.chat.id, 'ru_mnemonics: '+ts.google(self.mnemonics, from_language='en', to_language='ru',timeout=10))
    
    def make_ind_default(self):
        self.ind_meaning=0
        self.ind_reading=1

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------

class kan:
    def __init__(self, lvl, pic, meaning, reading, mnemonics='obvious', stage=0, previous_time=time.time()):
        self.previous_time=previous_time
        self.meaning=meaning #LIST
        self.reading=reading #LIST
        self.lvl=lvl
        self.ind_meaning=0
        self.ind_reading=0
        self.stage=stage
        self.pic=pic
        self.mnemonics=mnemonics
        self.type="KANJI"  
    
    def show(self,message,translate=False):
        photo = open(self.pic, 'rb')
        bot.send_message(message.chat.id, 'meanings: '+', '.join(self.meaning))
        bot.send_message(message.chat.id, 'readings: '+', '.join(self.reading))
        bot.send_photo(message.chat.id, photo)
        bot.send_message(message.chat.id, 'mnemonics: '+self.mnemonics)
        if translate:
            bot.send_message(message.chat.id, 'ru_mnemonics: '+ts.google(self.mnemonics, from_language='en', to_language='ru',timeout=10))

    def make_ind_default(self):
        self.ind_meaning=0
        self.ind_reading=0 

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------
class voc:
    def __init__(self, lvl, pic, meaning, reading, mnemonics='obvious',stage=0,previous_time=time.time()):
        self.previous_time=previous_time
        self.stage=stage
        self.ind_meaning=0
        self.ind_reading=0
        self.lvl=lvl
        self.pic=pic
        self.mnemonics=mnemonics
        self.meaning=meaning #LIST
        self.reading=reading #LIST
        self.type="VOCABULARY"  
    
    def show(self,message,translate=False):
        photo = open(self.pic, 'rb')
        bot.send_message(message.chat.id, 'meanings: '+', '.join(self.meaning))
        bot.send_message(message.chat.id, 'readings: '+', '.join(self.reading))
        bot.send_photo(message.chat.id, photo)
        bot.send_message(message.chat.id, 'mnemonics: '+self.mnemonics)
        if translate:
            bot.send_message(message.chat.id, 'ru_mnemonics: '+ts.google(self.mnemonics, from_language='en', to_language='ru',timeout=10))

    def make_ind_default(self):
        self.ind_meaning=0
        self.ind_reading=0

In [52]:
import telebot
from telebot import types

bot=telebot.TeleBot('5402875904:AAFkX76VxCNv1oUicVRguyOGQ6n-CRZHU1M')

user_dict={}
DIC_RAD,DIC_KAN,DIC_VOC,DIC_ALL,DIC_NAMES=FIT(1)
DIC_RAD,DIC_KAN,DIC_VOC,DIC_ALL,DIC_NAMES=FIT(4,DIC_RAD,DIC_KAN,DIC_VOC,DIC_ALL,DIC_NAMES)
A=wanikani(DIC_RAD,DIC_KAN,DIC_VOC,DIC_ALL,DIC_NAMES,4,int(1))

@bot.message_handler()
def ok(message):
    a,b=message.text.split('_')
    A.DIC_NAMES[(a,b)].show(message,True)
    




bot.infinity_polling()

2022-06-29 23:51:34,520 (__init__.py:574 MainThread) ERROR - TeleBot: "Infinity polling exception: not enough values to unpack (expected 2, got 1)"
2022-06-29 23:51:34,523 (__init__.py:576 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/telebot/__init__.py", line 570, in infinity_polling
    self.polling(non_stop=True, timeout=timeout, long_polling_timeout=long_polling_timeout,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/telebot/__init__.py", line 621, in polling
    self.__threaded_polling(non_stop=non_stop, interval=interval, timeout=timeout, long_polling_timeout=long_polling_timeout,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/telebot/__init__.py", line 695, in __threaded_polling
    raise e
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-pac

In [51]:
A.DIC_NAMES.keys()

dict_keys([('rad', 'mouth'), ('rad', 'river'), ('rad', 'power'), ('rad', 'two'), ('rad', 'person'), ('rad', 'gun'), ('rad', 'stick'), ('rad', 'big'), ('rad', 'lid'), ('rad', 'fins'), ('rad', 'toe'), ('rad', 'drop'), ('rad', 'enter'), ('rad', 'mountain'), ('rad', 'prison'), ('rad', 'cross'), ('rad', 'tree'), ('rad', 'ground'), ('rad', 'barb'), ('rad', 'woman'), ('rad', 'seven'), ('rad', 'construction'), ('rad', 'sun'), ('rad', 'leaf'), ('rad', 'slide'), ('rad', 'nine'), ('kan', 'above'), ('kan', 'ten'), ('kan', 'mouth'), ('kan', 'river'), ('kan', 'three'), ('kan', 'power'), ('kan', 'two'), ('kan', 'person'), ('kan', 'big'), ('kan', 'eight'), ('kan', 'enter'), ('kan', 'mountain'), ('kan', 'one'), ('kan', 'woman'), ('kan', 'seven'), ('kan', 'construction'), ('kan', 'nine'), ('kan', 'below'), ('voc', 'ten'), ('voc', 'alone'), ('voc', 'mouth'), ('voc', 'river'), ('voc', 'size'), ('voc', 'three'), ('voc', 'one thing'), ('voc', 'up'), ('voc', 'artificial'), ('voc', "with all one's strength"),

In [43]:
class wanikani:
    def __init__(self,DIC_RAD,DIC_KAN,DIC_VOC,DIC_ALL,DIC_NAMES,LVL=1,review_from=1):
        self.lvl=LVL
        self.review_from=review_from
        self.DIC_ALL=DIC_ALL
        self.DIC_RAD=DIC_RAD
        self.DIC_KAN=DIC_KAN
        self.DIC_VOC=DIC_VOC
        self.DIC_NAMES=DIC_NAMES

        self.ind_rad=1
        self.ind_kan=0
        self.ind_voc=0
        
    def info(self):
        c=0
        for i in self.DIC_KAN[self.lvl]:
            c = c+1 if i.stage>4 else c
        return ('CURRENT LVL: '+str(self.lvl)+'. PROGRESS: '+str(c)+" KANJI-GURU'S OUT OF "+str(len(self.DIC_KAN[self.lvl])),\
'NUM OF RADS IN THE LVL: '+str(len(self.DIC_RAD[self.lvl]))+'. KANS: '+str(len(self.DIC_KAN[self.lvl]))+'. VOCS: '+str(len(self.DIC_VOC[self.lvl])),\
                'ACTIVE LESSONS: '+str(self.count_lessons()),'ACTIVE REVIEWS: '+str(self.count_reviews()))